In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
import tensorflow.lite as tflite

import glob


# Parameters
dataset_url = r'C:\Users\abudh\Desktop\CropWatch\Test5'
batch_size = 4
img_height = int(2160 / 4)
img_width = int(1620 / 4)

validation_split = 0.2
rescale = 1.0 / 255
no_epochs = 100

# Data Augmentation and Loading
datagen = ImageDataGenerator(
    rescale=rescale,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=0.2,
    fill_mode='nearest',
    validation_split=validation_split
)

train_dataset = datagen.flow_from_directory(
    directory=dataset_url,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset="training",
    class_mode='categorical'
)

test_dataset = datagen.flow_from_directory(
    directory=dataset_url,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset="validation",
    class_mode='categorical'
)

# Compute class weights
class_indices = train_dataset.class_indices
class_labels = list(class_indices.keys())
class_indices = list(class_indices.values())
class_counts_dict = {class_name: len(glob.glob(os.path.join(dataset_url, class_name, '*'))) for class_name in class_labels}
class_counts = np.array([class_counts_dict[class_name] for class_name in class_labels])
weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array(class_indices),
    y=np.array([train_dataset.classes[i] for i in range(len(train_dataset.classes))])
)
class_weight_dict = dict(zip(class_labels, weights))

# Define the ResNet50 model
def identity_block(X, f, filters, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X, training=training)
    return X

def convolutional_block(X, f, filters, s=2, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

def ResNet50(input_shape=(img_height, img_width, 3), classes=len(class_labels)):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])
    X = AveragePooling2D(pool_size=(2, 2), name='avg_pool')(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer=glorot_uniform(seed=0))(X)
    model = Model(inputs=X_input, outputs=X)
    return model

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=15,
    verbose=1,
    min_lr=1e-6
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,
    verbose=1,
    restore_best_weights=True
)

model_path = r'C:\Users\abudh\Desktop\CropWatch'


# Corrected file paths to end with `.weights.h5`
dynamic_checkpoint_callback = ModelCheckpoint(
    filepath=model_path + r'\Training_model_epoch_13_08{epoch:02d}.weights.h5',  # Use .weights.h5 extension
    save_freq='epoch',
    save_weights_only=True,
    verbose=1
)

best_model_checkpoint_callback = ModelCheckpoint(
    filepath=model_path + r'\best_weights_13_08.weights.h5',  # Use .weights.h5 extension
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Compile and train the model
model = ResNet50(input_shape=(img_height, img_width, 3), classes=len(class_labels))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_dataset,
    epochs=no_epochs,
    validation_data=test_dataset,
    class_weight=class_weight_dict,
    callbacks=[reduce_lr, early_stopping, dynamic_checkpoint_callback, best_model_checkpoint_callback]
)


Found 253 images belonging to 2 classes.
Found 62 images belonging to 2 classes.
Epoch 1/100


C:\Users\abudh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5766 - loss: 4.8166
Epoch 1: saving model to C:\Users\abudh\Desktop\CropWatch\Training_model_epoch_13_0801.weights.h5

Epoch 1: val_loss improved from inf to 0.69010, saving model to C:\Users\abudh\Desktop\CropWatch\best_weights_13_08.weights.h5
64/64 ━━━━━━━━━━━━━━━━━━━━ 539s 7s/step - accuracy: 0.5766 - loss: 4.8138 - val_accuracy: 0.7419 - val_loss: 0.6901 - learning_rate: 1.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.6224 - loss: 3.1782
Epoch 2: saving model to C:\Users\abudh\Desktop\CropWatch\Training_model_epoch_13_0802.weights.h5

Epoch 2: val_loss did not improve from 0.69010
64/64 ━━━━━━━━━━━━━━━━━━━━ 583s 9s/step - accuracy: 0.6218 - loss: 3.1884 - val_accuracy: 0.5645 - val_loss: 2.0763 - learning_rate: 1.0000e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6330 - loss: 2.6091
Epoch 3: saving model to C:\Users\abudh\Desktop\CropWatch\Training_model_epoch_13_0803.weights.h5

E

In [ ]:
# Save the model

model_path = r'C:\Users\abudh\Desktop\CropWatch\13_08_model.h5'
model.save(model_path)


# Convert model to TensorFlow Lite format
converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_model_path = r'C:\Users\abudh\Desktop\CropWatch\13_08_model.tflite'

with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model converted to TFLite format and saved at: {tflite_model_path}")


In [ ]:
# Perform inference with TFLite model (example)
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

def predict_image_tflite(image):
    image = np.expand_dims(image, axis=0)
    image = image.astype(np.float32)
    
    interpreter.set_tensor(input_details['index'], image)
    interpreter.invoke()
    
    output_data = interpreter.get_tensor(output_details['index'])
    return np.argmax(output_data, axis=1)



In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Function for predicting with TFLite model
def predict_image_tflite(image):
    image = np.expand_dims(image, axis=0)
    image = image.astype(np.float32)
    
    interpreter.set_tensor(input_details['index'], image)
    interpreter.invoke()
    
    output_data = interpreter.get_tensor(output_details['index'])
    return np.argmax(output_data, axis=1)

# Initialize lists for storing predictions and true labels
y_pred = []  # store predicted labels
y_true = []  # store true labels

# Iterate over the dataset
num_batches = 10  # Limit to a smaller number for faster feedback
for i, (image_batch, label_batch) in enumerate(test_dataset):
    if i >= num_batches:
        break
    # Append true labels
    y_true.extend(np.argmax(label_batch, axis=1))
    # Compute predictions with TFLite model
    preds = [predict_image_tflite(image) for image in image_batch]
    # Append predicted labels
    y_pred.extend(preds)

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Normalize the confusion matrix by converting counts to percentages
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Define class names for plotting
class_indices = train_dataset.class_indices  # Ensure this is a dictionary
class_names = list(class_indices.keys())

# Plot the normalized confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Normalized Confusion Matrix (Percentage)')
plt.show()

# Print number of correct predictions and false positives
correct_predictions = np.diag(cm)
total_correct = correct_predictions.sum()

false_positives = cm.sum(axis=0) - correct_predictions
total_false_positives = false_positives.sum()

print(f"Number of correct predictions per class: {correct_predictions}")
print(f"Total number of correct predictions: {total_correct}")
print(f"Number of false positives per class: {false_positives}")
print(f"Total number of false positives: {total_false_positives}")

# Print a sample of predictions and true labels for inspection
print(f"Sample true labels: {y_true[:10]}")
print(f"Sample predicted labels: {y_pred[:10]}")
